In [1]:
import os
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append('../src/')

%load_ext autoreload
%autoreload 2
# Importing our custom module(s)
import utils

In [3]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
augmented_train_dataset, train_dataset, val_or_test_dataset = utils.get_cifar10_datasets(dataset_path, n=100, tune=True, random_state=1001)
[label for image, label in val_or_test_dataset]

Files already downloaded and verified
Files already downloaded and verified


[tensor(0),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(2),
 tensor(2),
 tensor(3),
 tensor(3),
 tensor(4),
 tensor(4),
 tensor(5),
 tensor(5),
 tensor(6),
 tensor(6),
 tensor(7),
 tensor(7),
 tensor(8),
 tensor(8),
 tensor(9),
 tensor(9)]

# Tune hyperparameters

In [4]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10'
lr_0s = np.logspace(-1, -4, num=4)
ns = [10, 100, 1000, 10000, 50000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_torchvision'
prior_type = 'nonlearned'
random_states = [1001, 2001, 3001]
wandb_project = 'tuned_CIFAR-10'
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
count = -1
for lr_0, n, random_state, weight_decay in itertools.product(lr_0s, ns, random_states, weight_decays):
    count += 1
    model_name = f'{prior_type}_lr_0={lr_0}_n={n}_random_state={random_state}_weight_decay={weight_decay}'
    print(f'    "python ../src/CIFAR10_main.py --dataset_path=\'{dataset_path}\' --experiments_path=\'{experiments_path}\' --lr_0={lr_0} --model_name=\'{model_name}\' --n={n} --prior_path=\'{prior_path}\' --prior_type=\'{prior_type}\' --random_state={random_state} --tune --wandb --wandb_project=\'{wandb_project}\' --weight_decay={weight_decay}"')
print(count)

    "python ../src/CIFAR10_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10' --lr_0=0.1 --model_name='nonlearned_lr_0=0.1_n=10_random_state=1001_weight_decay=0.01' --n=10 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_torchvision' --prior_type='nonlearned' --random_state=1001 --tune --wandb --wandb_project='tuned_CIFAR-10' --weight_decay=0.01"
    "python ../src/CIFAR10_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10' --lr_0=0.1 --model_name='nonlearned_lr_0=0.1_n=10_random_state=1001_weight_decay=0.001' --n=10 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_torchvision' --prior_type='nonlearned' --random_state=1001 --tune --wandb --wandb_project='tuned_CIFAR-10' --weight_decay=0.001"
    "python ../src/CIFAR10_main.py --datas

In [5]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10'
lr_0s = np.logspace(-1, -4, num=4)
ns = [10, 100, 1000, 10000, 50000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_type = 'adapted'
random_states = [1001, 2001, 3001]
wandb_project = 'tuned_CIFAR-10'
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
count = -1
for lr_0, n, random_state, weight_decay in itertools.product(lr_0s, ns, random_states, weight_decays):
    count += 1
    model_name = f'{prior_type}_lr_0={lr_0}_n={n}_random_state={random_state}_weight_decay={weight_decay}'
    print(f'    "python ../src/CIFAR10_main.py --dataset_path=\'{dataset_path}\' --experiments_path=\'{experiments_path}\' --lr_0={lr_0} --model_name=\'{model_name}\' --n={n} --prior_path=\'{prior_path}\' --prior_type=\'{prior_type}\' --random_state={random_state} --tune --wandb --wandb_project=\'{wandb_project}\' --weight_decay={weight_decay}"')
print(count)

    "python ../src/CIFAR10_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10' --lr_0=0.1 --model_name='adapted_lr_0=0.1_n=10_random_state=1001_weight_decay=0.01' --n=10 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='adapted' --random_state=1001 --tune --wandb --wandb_project='tuned_CIFAR-10' --weight_decay=0.01"
    "python ../src/CIFAR10_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10' --lr_0=0.1 --model_name='adapted_lr_0=0.1_n=10_random_state=1001_weight_decay=0.001' --n=10 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='adapted' --random_state=1001 --tune --wandb --wandb_project='tuned_CIFAR-10' --weight_decay=0.001"
    "python ../src/CIFAR10_main.py --dataset_path='/cluste

In [6]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10'
lr_0s = np.logspace(-3, -4, num=2)
ns = [10, 100, 1000, 10000, 50000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_scales = np.logspace(0, 9, num=10)
prior_type = 'learned'
random_states = [1001, 2001, 3001]
wandb_project = 'tuned_CIFAR-10'
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
count = -1
for lr_0, n, prior_scale, random_state, weight_decay in itertools.product(lr_0s, ns, prior_scales, random_states, weight_decays):
    count += 1
    model_name = f'{prior_type}_lr_0={lr_0}_n={n}_prior_scale={prior_scale}_random_state={random_state}_weight_decay={weight_decay}'
    print(f'    "python ../src/CIFAR10_main.py --dataset_path=\'{dataset_path}\' --experiments_path=\'{experiments_path}\' --lr_0={lr_0} --model_name=\'{model_name}\' --n={n} --prior_path=\'{prior_path}\' --prior_scale={prior_scale} --prior_type=\'{prior_type}\' --random_state={random_state} --tune --wandb --wandb_project=\'{wandb_project}\' --weight_decay={weight_decay}"')
print(count)

    "python ../src/CIFAR10_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10' --lr_0=0.001 --model_name='learned_lr_0=0.001_n=10_prior_scale=1.0_random_state=1001_weight_decay=0.01' --n=10 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_scale=1.0 --prior_type='learned' --random_state=1001 --tune --wandb --wandb_project='tuned_CIFAR-10' --weight_decay=0.01"
    "python ../src/CIFAR10_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10' --lr_0=0.001 --model_name='learned_lr_0=0.001_n=10_prior_scale=1.0_random_state=1001_weight_decay=0.001' --n=10 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_scale=1.0 --prior_type='learned' --random_state=1001 --tune --wandb --wandb_project='tuned_CIFAR-10' --we

# Retrain with selected parameters

In [7]:
def get_df(path):
    df = pd.read_csv(path, index_col='Unnamed: 0')
    return df

def get_val_nll(df):
    return df.val_or_test_nll.values[-1]

def get_val_acc(df):
    return df.val_or_test_acc.values[-1]

def get_last_epoch(df):
    return df.iloc[-1]

def get_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays):
    columns = ['lr_0', 'n', 'prior_scale', 'prior_type', 'random_state', 'val_acc', 'weight_decay']
    df = pd.DataFrame(columns=columns)
    for n, prior_scale, random_state in itertools.product(ns, prior_scales, random_states):
        best_val_nll = np.inf
        best_hyperparameters = None
        for lr_0, weight_decay in itertools.product(lr_0s, weight_decays):
            if prior_scale:
                model_name = f'{prior_type}_lr_0={lr_0}_n={n}_prior_scale={prior_scale}_random_state={random_state}_weight_decay={weight_decay}'
            else:
                model_name = f'{prior_type}_lr_0={lr_0}_n={n}_random_state={random_state}_weight_decay={weight_decay}'
            path =  f'{experiments_path}/{model_name}.csv'
            val_nll = get_val_nll(get_df(path))
            val_acc = get_val_acc(get_df(path))
            if val_nll < best_val_nll: best_val_nll = val_nll; best_hyperparameters = [lr_0, n, prior_scale, prior_type, random_state, val_acc, weight_decay]
        df.loc[df.shape[0]] = best_hyperparameters
    return df

def get_best_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays):
    columns = ['lr_0', 'n', 'prior_scale', 'prior_type', 'random_state', 'val_acc', 'weight_decay']
    df = pd.DataFrame(columns=columns)
    for n, random_state in itertools.product(ns, random_states):
        best_val_nll = np.inf
        best_hyperparameters = None
        for lr_0, prior_scale, weight_decay in itertools.product(lr_0s, prior_scales, weight_decays):
            if prior_scale:
                model_name = f'{prior_type}_lr_0={lr_0}_n={n}_prior_scale={prior_scale}_random_state={random_state}_weight_decay={weight_decay}'
            else:
                model_name = f'{prior_type}_lr_0={lr_0}_n={n}_random_state={random_state}_weight_decay={weight_decay}'
            path =  f'{experiments_path}/{model_name}.csv'
            val_nll = get_val_nll(get_df(path))
            val_acc = get_val_acc(get_df(path))
            if val_nll < best_val_nll: best_val_nll = val_nll; best_hyperparameters = [lr_0, n, prior_scale, prior_type, random_state, val_acc, weight_decay]
        df.loc[df.shape[0]] = best_hyperparameters
    return df

def get_results(df, experiments_path):
    columns = ['n', 'prior_scale', 'prior_type', 'random_state', 'test_acc', 'test_loss', 'test_nll', 
           'test_prior', 'train_acc', 'train_loss', 'train_nll', 'train_prior']
    results = pd.DataFrame(columns=columns)
    for row_index, row in df.iterrows():
        if row.prior_scale:
            model_name = f'{row.prior_type}_lr_0={row.lr_0}_n={row.n}_prior_scale={row.prior_scale}_random_state={row.random_state}_weight_decay={row.weight_decay}'
        else:
            model_name = f'{row.prior_type}_lr_0={row.lr_0}_n={row.n}_random_state={row.random_state}_weight_decay={row.weight_decay}'
        path =  f'{experiments_path}/{model_name}.csv'
        last_epoch = get_last_epoch(get_df(path))
        results_row = [int(row.n), row.prior_scale, row.prior_type, int(row.random_state), 
                       last_epoch.val_or_test_acc, last_epoch.val_or_test_loss, 
                       last_epoch.val_or_test_nll, last_epoch.val_or_test_prior, 
                       last_epoch.train_acc, last_epoch.train_loss, 
                       last_epoch.train_nll, last_epoch.train_prior]
        results.loc[results.shape[0]] = results_row
    return results

In [8]:
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10'
lr_0s = np.logspace(-1, -4, num=4)
ns = [10, 100, 1000, 10000, 50000]
prior_scales = [None]
prior_type = 'nonlearned'
random_states = [1001, 2001, 3001]
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
nonlearned_hyperparameters = get_best_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays)
nonlearned_hyperparameters

,lr_0,n,prior_scale,prior_type,random_state,val_acc,weight_decay
0,0.0100,10,None,nonlearned,1001,0.0000,0.01000
1,0.0001,10,None,nonlearned,2001,0.0000,0.01000
2,0.0010,10,None,nonlearned,3001,0.0000,0.01000
3,0.0100,100,None,nonlearned,1001,0.7000,0.00000
4,0.0010,100,None,nonlearned,2001,0.5000,0.00000
5,0.0010,100,None,nonlearned,3001,0.6500,0.00000
6,0.0001,1000,None,nonlearned,1001,0.7550,0.00010
7,0.0100,1000,None,nonlearned,2001,0.8200,0.00001
8,0.0100,1000,None,nonlearned,3001,0.8650,0.00100
9,0.0100,10000,None,nonlearned,1001,0.9265,0.00100


In [9]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_CIFAR-10'
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_torchvision'
wandb_project = 'retrained_CIFAR-10_torchvision'
count = -1
for row_index, row in nonlearned_hyperparameters.iterrows():
    if row.prior_scale:
        model_name = f'{row.prior_type}_lr_0={row.lr_0}_n={row.n}_prior_scale={row.prior_scale}_random_state={row.random_state}_weight_decay={row.weight_decay}'
        if not os.path.exists(f'{experiments_path}/{model_name}.csv'):
            count += 1
            print(f'    "python ../src/CIFAR10_main.py --dataset_path=\'{dataset_path}\' --experiments_path=\'{experiments_path}\' --lr_0={row.lr_0} --model_name=\'{model_name}\' --n={row.n} --prior_path=\'{prior_path}\' --prior_scale={row.prior_scale} --prior_type=\'{row.prior_type}\' --random_state={row.random_state} --wandb --wandb_project=\'{wandb_project}\' --weight_decay={row.weight_decay}"')
    else:
        model_name = f'{row.prior_type}_lr_0={row.lr_0}_n={row.n}_random_state={row.random_state}_weight_decay={row.weight_decay}'
        if not os.path.exists(f'{experiments_path}/{model_name}.csv'):
            count += 1
            print(f'    "python ../src/CIFAR10_main.py --dataset_path=\'{dataset_path}\' --experiments_path=\'{experiments_path}\' --lr_0={row.lr_0} --model_name=\'{model_name}\' --n={row.n} --prior_path=\'{prior_path}\' --prior_type=\'{row.prior_type}\' --random_state={row.random_state} --wandb --wandb_project=\'{wandb_project}\' --weight_decay={row.weight_decay}"')
print(count)

-1


In [10]:
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10'
lr_0s = np.logspace(-1, -4, num=4)
ns = [10, 100, 1000, 10000, 50000]
prior_scales = [None]
prior_type = 'adapted'
random_states = [1001, 2001, 3001]
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
adapted_hyperparameters = get_best_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays)
adapted_hyperparameters

,lr_0,n,prior_scale,prior_type,random_state,val_acc,weight_decay
0,0.100,10,None,adapted,1001,0.0000,0.010000
1,0.100,10,None,adapted,2001,0.0000,0.010000
2,0.100,10,None,adapted,3001,0.0000,0.010000
3,0.001,100,None,adapted,1001,0.7000,0.000000
4,0.001,100,None,adapted,2001,0.5000,0.010000
5,0.001,100,None,adapted,3001,0.6500,0.000001
6,0.100,1000,None,adapted,1001,0.8350,0.010000
7,0.100,1000,None,adapted,2001,0.8150,0.010000
8,0.010,1000,None,adapted,3001,0.9100,0.000010
9,0.100,10000,None,adapted,1001,0.9335,0.010000


In [11]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_CIFAR-10'
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
wandb_project = 'retrained_CIFAR-10'
count = -1
for row_index, row in adapted_hyperparameters.iterrows():
    if row.prior_scale:
        model_name = f'{row.prior_type}_lr_0={row.lr_0}_n={row.n}_prior_scale={row.prior_scale}_random_state={row.random_state}_weight_decay={row.weight_decay}'
        if not os.path.exists(f'{experiments_path}/{model_name}.csv'):
            count += 1
            print(f'    "python ../src/CIFAR10_main.py --dataset_path=\'{dataset_path}\' --experiments_path=\'{experiments_path}\' --lr_0={row.lr_0} --model_name=\'{model_name}\' --n={row.n} --prior_path=\'{prior_path}\' --prior_scale={row.prior_scale} --prior_type=\'{row.prior_type}\' --random_state={row.random_state} --wandb --wandb_project=\'{wandb_project}\' --weight_decay={row.weight_decay}"')
    else:
        model_name = f'{row.prior_type}_lr_0={row.lr_0}_n={row.n}_random_state={row.random_state}_weight_decay={row.weight_decay}'
        if not os.path.exists(f'{experiments_path}/{model_name}.csv'):
            count += 1
            print(f'    "python ../src/CIFAR10_main.py --dataset_path=\'{dataset_path}\' --experiments_path=\'{experiments_path}\' --lr_0={row.lr_0} --model_name=\'{model_name}\' --n={row.n} --prior_path=\'{prior_path}\' --prior_type=\'{row.prior_type}\' --random_state={row.random_state} --wandb --wandb_project=\'{wandb_project}\' --weight_decay={row.weight_decay}"')
print(count)

-1


In [12]:
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_CIFAR-10'
lr_0s = np.logspace(-1, -4, num=4)
ns = [10, 100, 1000, 10000, 50000]
prior_scales = np.logspace(0, 9, num=10)
prior_type = 'learned'
random_states = [1001, 2001, 3001]
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
learned_hyperparameters = get_best_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays)
learned_hyperparameters

,lr_0,n,prior_scale,prior_type,random_state,val_acc,weight_decay
0,0.010,10,1.000000e+06,learned,1001,0.0000,0.010000
1,0.010,10,1.000000e+05,learned,2001,0.0000,0.010000
2,0.010,10,1.000000e+05,learned,3001,0.0000,0.010000
3,0.010,100,1.000000e+00,learned,1001,0.8500,0.000100
4,0.010,100,1.000000e+02,learned,2001,0.6000,0.000000
5,0.001,100,1.000000e+09,learned,3001,0.6500,0.000000
6,0.010,1000,1.000000e+00,learned,1001,0.8450,0.000001
7,0.100,1000,1.000000e+00,learned,2001,0.8600,0.000001
8,0.100,1000,1.000000e+03,learned,3001,0.8800,0.000000
9,0.010,10000,1.000000e+01,learned,1001,0.9355,0.001000


In [13]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_CIFAR-10'
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
wandb_project = 'retrained_CIFAR-10'
count = -1
for row_index, row in learned_hyperparameters.iterrows():
    if row.prior_scale:
        model_name = f'{row.prior_type}_lr_0={row.lr_0}_n={row.n}_prior_scale={row.prior_scale}_random_state={row.random_state}_weight_decay={row.weight_decay}'
        if not os.path.exists(f'{experiments_path}/{model_name}.csv'):
            count += 1
            print(f'    "python ../src/CIFAR10_main.py --dataset_path=\'{dataset_path}\' --experiments_path=\'{experiments_path}\' --lr_0={row.lr_0} --model_name=\'{model_name}\' --n={row.n} --prior_path=\'{prior_path}\' --prior_scale={row.prior_scale} --prior_type=\'{row.prior_type}\' --random_state={row.random_state} --wandb --wandb_project=\'{wandb_project}\' --weight_decay={row.weight_decay}"')
    else:
        model_name = f'{row.prior_type}_lr_0={row.lr_0}_n={row.n}_random_state={row.random_state}_weight_decay={row.weight_decay}'
        if not os.path.exists(f'{experiments_path}/{model_name}.csv'):
            count += 1
            print(f'    "python ../src/CIFAR10_main.py --dataset_path=\'{dataset_path}\' --experiments_path=\'{experiments_path}\' --lr_0={row.lr_0} --model_name=\'{model_name}\' --n={row.n} --prior_path=\'{prior_path}\' --prior_type=\'{row.prior_type}\' --random_state={row.random_state} --wandb --wandb_project=\'{wandb_project}\' --weight_decay={row.weight_decay}"')
print(count)

-1


In [14]:
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_CIFAR-10'
nonlearned_results = get_results(nonlearned_hyperparameters, experiments_path)
adapted_results = get_results(adapted_hyperparameters, experiments_path)
learned_results = get_results(learned_hyperparameters, experiments_path)
results = pd.concat([nonlearned_results, adapted_results, learned_results])
results

,n,prior_scale,prior_type,random_state,test_acc,test_loss,test_nll,test_prior,train_acc,train_loss,train_nll,train_prior
0,10,NaN,nonlearned,1001,0.1317,3.200835,3.200835,0.000000,1.00000,1.793888e-02,0.017939,0.000000e+00
1,10,NaN,nonlearned,2001,0.2621,2.097741,2.097741,0.000000,1.00000,4.409529e-03,0.004410,0.000000e+00
2,10,NaN,nonlearned,3001,0.2669,2.124110,2.124110,0.000000,1.00000,1.155219e-03,0.001155,0.000000e+00
3,100,NaN,nonlearned,1001,0.5301,1.753938,1.753938,0.000000,1.00000,1.750569e-05,0.000018,0.000000e+00
4,100,NaN,nonlearned,2001,0.5987,1.273124,1.273124,0.000000,1.00000,2.410493e-04,0.000241,0.000000e+00
5,100,NaN,nonlearned,3001,0.5575,1.404902,1.404902,0.000000,1.00000,2.823434e-04,0.000282,0.000000e+00
6,1000,NaN,nonlearned,1001,0.7818,0.689700,0.689700,0.000000,0.99900,2.121432e-02,0.021214,0.000000e+00
7,1000,NaN,nonlearned,2001,0.8181,0.790316,0.790316,0.000000,1.00000,5.853714e-06,0.000006,0.000000e+00
8,1000,NaN,nonlearned,3001,0.8275,0.655546,0.655546,0.000000,1.00000,6.708927e-05,0.000067,0.000000e+00
9,10000,NaN,nonlearned,1001,0.9372,0.226423,0.226423,0.000000,1.00000,1.199188e-04,0.000120,0.000000e+00


In [15]:
grouped_results = results.groupby(['n', 'prior_type']).agg(lambda x: tuple(x))
columns = ['test_acc', 'train_acc', 'test_nll', 'train_nll']
for column in columns:
    grouped_results[f'{column}_mean'] = grouped_results[column].apply(lambda item: np.mean(item))
    grouped_results[f'{column}_std'] = grouped_results[column].apply(lambda item: np.std(item))
    grouped_results[f'{column}_min'] = grouped_results[column].apply(lambda item: np.min(item))
    grouped_results[f'{column}_max'] = grouped_results[column].apply(lambda item: np.max(item))
grouped_results = grouped_results.reset_index()
grouped_results

,n,prior_type,prior_scale,random_state,test_acc,test_loss,test_nll,test_prior,train_acc,train_loss,...,train_acc_min,train_acc_max,test_nll_mean,test_nll_std,test_nll_min,test_nll_max,train_nll_mean,train_nll_std,train_nll_min,train_nll_max
0,10,adapted,"(nan, nan, nan)","(1001, 2001, 3001)","(0.2339000254869461, 0.175599992275238, 0.2116...","(2.2504510540008558, 2.351904470062255, 2.3314...","(2.244244303131104, 2.346000033950806, 2.32525...","(0.0062067494727671, 0.0059044128283858, 0.006...","(1.0000001192092896, 1.0000001192092896, 1.000...","(0.0067454506643116, 0.0063801310025155, 0.006...",...,1.00000,1.00000,2.305165,0.043903,2.244244,2.346000,0.000497,0.000029,0.000476,0.000539
1,10,learned,"(1000000.0, 100000.0, 100000.0)","(1001, 2001, 3001)","(0.1660000085830688, 0.1442999988794326, 0.160...","(6093.6448609375, 3447.17701171875, 3447.10242...","(2.317222708129884, 2.4165171756744384, 2.3419...","(-6091.327636718749, -3444.760498046871, -3444...","(1.0000001192092896, 1.0000001192092896, 1.000...","(6091327.5, 3444760.5, 3444760.5)",...,1.00000,1.00000,2.358558,0.042207,2.317223,2.416517,0.017839,0.000112,0.017696,0.017969
2,10,nonlearned,"(nan, nan, nan)","(1001, 2001, 3001)","(0.1316999942064285, 0.2621000111103058, 0.266...","(3.200834867477418, 2.0977412963867184, 2.1241...","(3.200834867477418, 2.0977412963867184, 2.1241...","(0.0, 0.0, 0.0)","(1.0000001192092896, 1.0000001192092896, 1.000...","(0.017938882112503, 0.0044095288030803, 0.0011...",...,1.00000,1.00000,2.474229,0.513901,2.097741,3.200835,0.007835,0.007267,0.001155,0.017939
3,100,adapted,"(nan, nan, nan)","(1001, 2001, 3001)","(0.5476000308990479, 0.5982000231742859, 0.555...","(1.4966200017929077, 1.3283599117279048, 1.406...","(1.4966200017929077, 1.3010373809814455, 1.406...","(0.0, 0.0273224879056215, 4.0377854020334806e-06)","(1.0000001192092896, 1.0000001192092896, 1.000...","(0.0002863011031877, 0.0286185126751661, 0.000...",...,1.00000,1.00000,1.401535,0.079938,1.301037,1.496620,0.000625,0.000475,0.000286,0.001296
4,100,learned,"(1.0, 100.0, 1000000000.0)","(1001, 2001, 3001)","(0.5631999969482422, 0.5616999864578247, 0.556...","(-543.6378310546875, -491.9241127929687, 14204...","(1.38884101638794, 1.365194254684448, 1.398413...","(545.0266723632818, 493.28930664062415, -14203...","(1.0000001192092896, 1.0000001192092896, 1.000...","(-54502.6640625, -49328.9296875, 1420327.5)",...,1.00000,1.00000,1.384149,0.013961,1.365194,1.398413,0.001154,0.000615,0.000285,0.001613
5,100,nonlearned,"(nan, nan, nan)","(1001, 2001, 3001)","(0.5300999879837036, 0.5986999869346619, 0.557...","(1.7539382343292234, 1.2731240596771238, 1.404...","(1.7539382343292234, 1.2731240596771238, 1.404...","(0.0, 0.0, 0.0)","(1.0000001192092896, 1.0000001192092896, 1.000...","(1.7505688447272405e-05, 0.0002410493034403, 0...",...,1.00000,1.00000,1.477322,0.202861,1.273124,1.753938,0.000180,0.000116,0.000018,0.000282
6,1000,adapted,"(nan, nan, nan)","(1001, 2001, 3001)","(0.8163000345230103, 0.820099949836731, 0.8209...","(0.6971136625289917, 0.6787592329025266, 0.783...","(0.6228910679817198, 0.6203525841712954, 0.782...","(0.0742225944995879, 0.0584066323935986, 0.000...","(1.0000001192092896, 1.0000001192092896, 1.000...","(0.0751252514123916, 0.0592542142570018, 0.000...",...,1.00000,1.00000,0.675372,0.076022,0.620353,0.782873,0.000586,0.000410,0.000006,0.000903
7,1000,learned,"(1.0, 1.0, 1000.0)","(1001, 2001, 3001)","(0.8384000062942505, 0.8154000043869019, 0.820...","(-544.4517595703124, -544.3486912109374, -165....","(0.5736306971549986, 0.6764535427093508, 0.722...","(545.025390624999, 545.0251464843751, 165.8978...","(1.0000001192092896, 1.0000001192092896, 1.000...","(-5450.25384375, -5450.251214843751, -1658.978...",...,1.00000,1.00000,0.657566,0.062271,0.573631,0.722613,0.000213,0.000038,0.000175,0.000265
8,1000,nonlearned,"(nan, nan, nan)","(1001, 2001, 3001)","(0.7818000316619873, 0.8181000351905823, 0.827...","(0.6896996079444885, 0.7903160516738894, 0.655..

In [16]:
grouped_results[['n', 'prior_type', 'test_acc_mean', 'test_acc_min', 'test_acc_max']]

,n,prior_type,test_acc_mean,test_acc_min,test_acc_max
0,10,adapted,0.207067,0.1756,0.2339
1,10,learned,0.156900,0.1443,0.1660
2,10,nonlearned,0.220233,0.1317,0.2669
3,100,adapted,0.567267,0.5476,0.5982
4,100,learned,0.560433,0.5564,0.5632
5,100,nonlearned,0.562100,0.5301,0.5987
6,1000,adapted,0.819133,0.8163,0.8210
7,1000,learned,0.824867,0.8154,0.8384
8,1000,nonlearned,0.809133,0.7818,0.8275
9,10000,adapted,0.943433,0.9410,0.9447


In [17]:
grouped_results[['n', 'prior_type', 'test_nll_mean', 'test_nll_min', 'test_nll_max']]

,n,prior_type,test_nll_mean,test_nll_min,test_nll_max
0,10,adapted,2.305165,2.244244,2.346000
1,10,learned,2.358558,2.317223,2.416517
2,10,nonlearned,2.474229,2.097741,3.200835
3,100,adapted,1.401535,1.301037,1.496620
4,100,learned,1.384149,1.365194,1.398413
5,100,nonlearned,1.477322,1.273124,1.753938
6,1000,adapted,0.675372,0.620353,0.782873
7,1000,learned,0.657566,0.573631,0.722613
8,1000,nonlearned,0.711854,0.655546,0.790316
9,10000,adapted,0.184208,0.180804,0.186832
